In [1]:
import numpy as np
# import cvxpy as cp
import matplotlib.pyplot as plt
import matplotlib
%matplotlib qt
import seaborn as sns
import time
import scipy.stats

def matprint(mat, fmt="g"):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")

In [2]:
'''
using 3 types of material, measurement points intake, after a, after b

'''

def dynamics(A, B, x, u):
    x_new = A @ x + B @ u
    x_new[x_new < 0] = 0  #clip material quantities to zero
    return x_new
    
    
def problem_constructor_LP(num_materials = 4, param = 1, d=1, set_params = False):
    
    '''
    x order: (assuming 4 materials)
    initial state (4)
    stage 1 (4)
    stage 2 (4)
    stage 3/out material 4 (4) 
    effort mat 1
    effort mat 2
    effort mat 3
    belt speed
    '''
    nx = 0
    nu = 0
    # nx += num_materials  # compisition of feed pile, normalized to 1
    nx += num_materials ** 2 # quantity of each material at each stage (intake, stage 1, stage ..., stage num_material-1)
    nx += num_materials - 1 # effort at each positive sort station
    # nx += 1 # belt speed (really, how much to take from feed pile each time step)
    
    # nu = num_materials - 1 # how much to increase or decrease effort at each positive sort station
    nu += 1 #how much to increase or decrease belt speed
    
    x = np.zeros((nx))
    u = np.zeros((nu))
    
    A = np.zeros((nx, nx))
    B = np.zeros((nx, nu))
    Q_vect_plenty = np.zeros((nx))
    R_vect = np.ones((nu))
    Q_vect_tapped = np.zeros((nx))
    
    
    ## assemble a and b matrices
    
    priceA = 66 #$/ton HDPE
    priceB = 13 #$/ton pet
    priceC = 5 #$/ton pp
    priceD = 1 #$/ton 3-7 comingled
    
    # Q_vect_plenty[-1] = priceC
    # Q_vect_plenty[-2] = priceB
    # Q_vect_plenty[-3] = priceA
    # Q_vect_plenty[-4] = priceD-priceA
    # Q_vect_plenty[-5] = priceD-priceB
    # Q_vect_plenty[-6] = priceD-priceC
    # Q_vect_plenty[-7] = priceD
    
    Q_vect_tapped[0] = priceA
    Q_vect_tapped[5] = priceB
    Q_vect_tapped[10] = priceC
    Q_vect_tapped[-4] = priceD
    Q_vect_tapped[-5] = priceD-priceC
    Q_vect_tapped[-6] = priceD-priceB
    Q_vect_tapped[-7] = priceD - priceA
    
    # R_vect *= -1
    # R_vect[-1] = 0
    
    # Q = np.diag(Q_vect)
    # R = np.diag(R_vect)
    
    
    
    
    # feedA = 0.5
    # feedB = 1
    # feedC = .8
    # feedD = 0.2
    feedA = 1 * param
    feedB = 1
    feedC = 1
    feedD = 1
    
    
    sum = feedA + feedB + feedC + feedD
    feedA = feedA/sum
    feedB = feedB/sum
    feedC = feedC/sum
    feedD = feedD/sum
    
    B[0, -1] = feedA
    B[1, -1] = feedB
    B[2, -1] = feedC
    B[3, -1] = feedD
    
    
    
    # top part of last column of b encodes feed pile ratios, u[-1] is how much feed pile to put in first few rows of x
    
    
    # A[-1, -1] = 1 # belt speed no state transition
    # B[-1, -1] = 1 # belt speed increase or decrease equal to last element of u
    
    ## feed pile stays the same
    # for i in range(num_materials):
    #     A[i, i] = 1
    
    
    for i in range(num_materials-1):
        
        # move stuff down the conveyor belt (first station draws from feed pile)
        row_start = (i+1)*num_materials
        col_start = i*num_materials
        for j in range(num_materials):
            Ar = row_start+j
            Ac = col_start+j
            A[Ar, Ac] = 1
        # remove material from flow at sorting stations
        col_sort = num_materials**2 + i
        A[row_start + i, col_sort] = -1
        A[-1*(i+1), -1*(i+1)] = 1
        
    
    # if num_materials == 4 and set_params == True:
    #     print()
    # else:
    #     for i in range(num_materials):
    #         B[i, -1] = np.random()
    #         print('feed pile material a ')
            
    # r_start = num_materials **2 -1
            
    # for i in range(num_materials-1):
        
    #     # adjust sorting effrots as per controls
        
    #     B[r_start+i, i] = 1
    
    return A, B, x, u, Q_vect_tapped, R_vect

def traj_cost(u_traj, x_init, A, B, Q, R):
    #inputs: vector of delta speeds, initial x vector
    
    N = u_traj.shape[0]
    x_traj = np.zeros((N+1, x_init.shape[0]))
    x_traj[0] = x_init
    cost = 0
    x_mask = 1000000000*np.ones_like(x_init)
    x_mask[0] = 1
    x_mask[5] = 1
    x_mask[10] = 1
    
    for k in range(N):
        # print(A.shape)
        # print(B.shape)
        # print(x_traj[k].shape)
        # print(u_traj[k])
        # print(dynamics(A, B, x_traj[k], u_traj[k]))
        x_traj[k+1] = dynamics(A, B, x_traj[k], u_traj[k])
        cost += Q @ np.minimum(x_traj[k+1], x_mask) + R @ u_traj[k]
        # print(cost, k)
        
    return cost
    
    #output: scalar cost
    

In [3]:
u_traj =np.ones((4, 1))

delta = 0.01
deltas = delta * np.diag(np.ones_like(u_traj).T[0])
base_cost = traj_cost(u_traj, x, A, B, Q, R)
diff = np.zeros_like(u_traj)

for i in range(u_traj.shape[0]):
        diff[i] = (traj_cost((u_traj.T[0]+deltas[i])[:, np.newaxis], x, A, B, Q, R) - base_cost)/delta
        print("diffi: ", diff[i])
        
# traj_cost((u_traj.T[0]+deltas[0])[:, np.newaxis], x, A, B, Q, R)

NameError: name 'x' is not defined

In [4]:
# loop structure

# init problem
A, B, x, u, Q, R = problem_constructor_LP(4, param = 3)

    # initial conditions:
x[-3] = 1
x[-2] = 1
x[-1] = 1
R[-1] = -0.1
u_traj =1*np.ones((8, 1))

# set starting state

start_time = time.time()
# get gradients from finite diff
delta = 0.01
deltas = delta * np.diag(np.ones_like(u_traj).T[0])

lr = 0.1
diff = 10*np.ones_like(u_traj)
for j in range(100):
    base_cost = traj_cost(u_traj, x, A, B, Q, R)
    plt.scatter(u_traj[0], base_cost, label="Iteration: "+str(j))
    
    # print("\n\nbase_cost: ", base_cost)
    # print('u_traj', u_traj)
    if np.linalg.norm(diff) < 0.01:
        # print("converged in all values!")
        break
    for i in range(u_traj.shape[0]):
        if np.linalg.norm(diff[i]) > 1.01:
            diff[i] = (traj_cost((u_traj.T[0]+deltas[i])[:, np.newaxis], x, A, B, Q, R) - base_cost)/delta
        else:
            diff[i] = 0
            # print('converged: , ', i)
        
        # print("diffi: ", diff[i])
        
    u_traj += lr*diff
    # print('Cost', base_cost)
    # print('update:', diff)
    
    lr*=0.5
plt.scatter(u_traj[0], base_cost, s=400,color='r', marker=(5, 1))
print("Converged! solve time (sec): ", time.time() - start_time, 'iters: ', j)
# plt.legend()
plt.xlabel('Belt Speed')
plt.ylabel('Objective Function Value')
plt.show()
print(u_traj)
# update decision variables



Converged! solve time (sec):  0.5391626358032227 iters:  99
[[1.99534309]
 [1.99534309]
 [1.99534309]
 [1.99534309]
 [1.99534309]
 [4.88      ]
 [4.71333333]
 [4.285     ]]


In [5]:
deltas

array([[0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.01, 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.01, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.01]])

In [5]:
def solve_next_step(x, u_traj_last, A, B, Q, R):
    
    u_traj = np.copy(u_traj_last)
    delta = 0.01
    deltas = delta * np.diag(np.ones_like(u_traj).T[0])

    lr = 0.1
    diff = 10*np.ones_like(u_traj)
    for j in range(1000):
        base_cost = traj_cost(u_traj, x, A, B, Q, R)
        # if np.linalg.norm(diff) < 0.01:
        #     break
        for i in range(u_traj.shape[0]):
            if np.linalg.norm(diff[i]) > 1.01:
                diff[i] = (traj_cost((u_traj.T[0]+deltas[i])[:, np.newaxis], x, A, B, Q, R) - base_cost)/delta
            else:
                diff[i] = 0

        u_traj += lr*diff * np.random.random(size=1)
        
        
        lr*0.9
    
    return u_traj

In [286]:
B

array([[0.25],
       [0.25],
       [0.25],
       [0.25],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.  ]])

In [38]:
ax = sns.heatmap(mat, linewidth=0.5, xticklabels = np.round(exes, 2), yticklabels = np.round(exes_log, 2), cbar_kws={'label': 'Steady-State Objective Function Value'})
plt.xlabel('Belt Speed')
plt.ylabel('Quantity of HDPE in Flow')
plt.show()


In [3]:
# code structure:
n_materials = 4

horizon = 100

infeed = np.ones((100, 4))

num_hits = 10
x = np.arange(0, 100, 1)

hits_1 = np.random.randint(0, 100, num_hits)
hits_2 = np.random.randint(0, 100, num_hits)
hits_3 = np.random.randint(0, 100, num_hits)
hits_4 = np.random.randint(0, 100, num_hits)

hit_inds = np.random.randint(0, 100, (num_hits, n_materials))
for hit in range(num_hits):
    for i in range(n_materials):
        infeed[:, i] += scipy.stats.norm.pdf(x, hit_inds[hit, i], 10)*10000
        

infeed_norm = np.sum(infeed, axis=1, keepdims=True)

infeed = infeed/infeed_norm

plt.plot(infeed[:, 0])
plt.plot(infeed[:, 1])
plt.plot(infeed[:, 2])
plt.plot(infeed[:, 3])

# generate infeed, plot it in this cell

In [8]:
'''
loop through infeed array, at each time step do the following:

Create A, B matrices from log

u_traj = solve_for_next_step(x_current, u_traj_last, A, B, Q, R)

roll out dynamics one step using u_traj

'''
np.random.seed(1)

A, B, x, u, Q, R = problem_constructor_LP(4, param = 3)

acc_cost_opt = np.zeros((horizon+1))
acc_cost_base = np.zeros((horizon+1))
x_mask = 1000000000*np.ones_like(x)
x_mask[0] = 1
x_mask[5] = 1
x_mask[10] = 1
U = np.zeros((horizon))
U_base = np.zeros((horizon))

x_base = np.copy(x)
u_base = 4*np.ones((1, 1))

u_traj = 4*np.ones((5, 1))
step = 0
solve_times = np.zeros((infeed.shape[0]))

for i in infeed:
    B[:4, 0] = i
    solve_start = time.time()
    u_traj_next = solve_next_step(x, u_traj, A, B, Q, R)
    solve_times[step] = time.time() - solve_start
    x = dynamics(A, B, x, u_traj_next[0])
    x_base = dynamics(A, B, x_base, u_base[0])
    u_traj = u_traj_next
    U[step] = u_traj[0]
    U_base[step] = u_base[0]
    if step > 4:
        acc_cost_opt[step+1] = acc_cost_opt[step] + Q @ np.minimum(x, x_mask) + R @ u_traj[0]
        acc_cost_base[step+1] = acc_cost_base[step] + Q @ np.minimum(x_base, x_mask) + R @ u_base[0]
    step += 1
    
    
    

In [279]:
fig, axes = plt.subplots(3)

axes[0].plot(infeed[5:, 0], label = 'Material A')
axes[0].plot(infeed[5:, 1], label = 'Material B')
axes[0].plot(infeed[5:, 2], label = 'Material C')
axes[0].plot(infeed[5:, 3], label = 'Material D')

# _, cost_fig = plt.figure()
axes[1].plot(acc_cost_opt[5:], label = 'Total Accumulated Reward - Optimized')
axes[1].plot(acc_cost_base[5:], label = 'Total Accumulated Reward - Current (no control)')
# _, speed_fig = plt.figure()
axes[2].plot(U[5:], label = 'Belt Speed')
axes[2].plot(U_base[5:], label = 'Belt Speed - base')


axes[0].legend()
# axes[0].show()

axes[1].legend()
# cost_fig.show()

axes[2].legend()
# speed_fig.show()
plt.show()

In [11]:
plt.hist(solve_times, bins='auto')
plt.xlabel('Time to convergence (seconds)')
plt.ylabel('Percent of solves')


Text(46.972222222222214, 0.5, 'Percent of solves')

In [214]:
x_base

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [215]:
u_traj[0]

array([4.])